In [1]:
from pytrips.ontology import get_ontology as tripsont
from pytrips.structures import TripsType
from nltk.corpus.reader.wordnet import Synset
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')
semcor_ic = wordnet_ic.ic('ic-semcor.dat')
from collections import defaultdict
from collections import namedtuple
import random

import math
from scipy.stats import spearmanr, pearsonr

nouns = list(wn.all_synsets(pos=wn.NOUN))
num = 100
noun_sample = random.sample(nouns, num)

In [2]:
"""
Default node weights, can be overrided for variations
"""
_node_weights = defaultdict(lambda: 1)

_node_weights["fakeroot"] = 0

_dcache = {}


def _fallback(x, func, nonzero=False, fbs=-1):
    """
    Basic null checks for selecting a value from a list
    """
    if (len(x) == 0) or (sum(x) == 0 and nonzero):
        return fbs
    return func(x)

"""
Pick a value from a list
"""
ListStrategy = {
    'choose': lambda f: lambda b: lambda x: _fallback(x, f, fbs=-1),
    'min': lambda x: _fallback(x, min),
    'bmin': lambda b: lambda x: _fallback(x, min, fbs=b),
    'max': lambda x: _fallback(x, max),
    'bmax': lambda b: lambda x: _fallback(x, max, fbs=b),
    'average': lambda x: _fallback(x, lambda l: sum(l)/len(l), nonzero=True)
}


"""
Find the last common element between two lists
"""
def last_overlap(v1, v2, aligned=None):
    if not v1 or not v2:
        return aligned
    elif v1[0] != v2[0]:
        return aligned
    else:
        return last_overlap(v1[1:], v2[1:], aligned=v1[0])

class SemNode:
    def __init__(self, node):
        self._node = node
    
    def __eq__(self, other):
        if issubclass(type(other), SemNode):
            if type(other.content) == type(self.content):
                return other.content == self.content
        return False
    
    def __hash__(self):
        return hash(self.__repr__())
    
    def __repr__(self):
        return "<SemNode: {}>".format(self.content.__repr__())
    
    def __str__(self):
        return "<SemNode: {}>".format(self.name)
    
    @property
    def name(self):
        """
        return canonical name for node
        """
        return str(self.content)
    
    @property
    def adjacent(self, label=None):
        """
        Return connected elements by label.  Useful for following non-hypernym relations in wordnet
        Can be used to implement other connections.
        Override as necessary.
        """
        return []
    
    @property
    def root(self):
        """
        Check if the node is a root of some sort.  Override as necessary for different resources.
        """
        return not self.parents
    
    @property
    def parents(self):
        """
        Get all parents for a node.  Perform any cross-resource cutoffs.
        """
        pass
    
    @property
    def children(self):
        """
        Get all children for a node.  Perform any cross resource cutoffs.
        """
        pass
    
    @property
    def content(self):
        """
        Return the wrapped node.
        """
        return self._node
    
    @property
    def resource(self):
        """
        Return the name of the relevant resource
        """
        return "default"
    
    def weight(self, weights=None):
        """
        Get resource-based weight of node.  Pass a weight dictionary as necessary.
        TODO: add an argument to pass individual weights
        """
        if not weights:
            weights = _node_weights
        return weights[self.resource]
    
    @staticmethod
    def path_depth(path, weights=None):
        """
        Get the total depth of a path, defined as a list of nodes.
        Does not check validity of path
        """
    
        return sum([p.weight(weights=weights) for p in path])
    
    @staticmethod
    def depth(node, weights=None, strategy='max'):
        """
        Get the depth of a node to a root using a weight dictionary and selection strategy.
        Default is minimum depth from any root.
        """
        if strategy in _dcache:
            if node.content in _dcache:
                return _dcache[strategy][node.content]
        else:
            _dcache[strategy] = {}
        weighted = [SemNode.path_depth(p, weights=weights) for p in node.paths_to_root()]
        _dcache[strategy][node.content] = ListStrategy[strategy](weighted) - 1
        return _dcache[strategy][node.content]
    
    @staticmethod
    def make(node):
        """
        Make a node based on the input type.
        Should add parameter dictionary to pass on to children
        """
        if type(node) is TripsType:
            return TripsNode(node)
        elif type(node) is Synset:
            return WordNetNode(node)
        elif type(node) is str:
            return WordNode(node)
        elif type(node) is SemNode:
            return node
        else:
            return None
    
    def paths_to_root(self):
        """
        Find all paths to a root based on hierarchy rules.
        Some resources return only one (Trips), others may return multiple (WordNet)
        """
        if self.root:
            return [[self]]
        res = []
        for c in self.parents:
            ptrs = c.paths_to_root()
            res.extend([t + [self] for t in ptrs if self not in t])
        return res
    
    def lcs_set(self, other):
        """
        Find the set of Lowest Common Subsumers for a node.  
        Some resources have only one (Trips) other can have multiple (WordNet)
        """
        lcs = [last_overlap(p,q) for p in self.paths_to_root() for q in other.paths_to_root()]
        filtered = [x for x in lcs if x]
        if not filtered:
            return [TripsNode(tripsont()["root"])]
        return filtered
    
    def wupalmer(self, other, weights=None, depth_strategy='min', lcs_strategy='max'):
        """
        return cross-wupalmer measure using provided weights, depth_strategy and lcs_strategy
        depth_strategy: Choose max, min, or average depth over all paths
        lcs_strategy: Choose max, min, or average depth of lcs over all alternatives
        """
        if not issubclass(type(other), SemNode):
            other = SemNode.make(other) # this would break passing in an arbitrary maker object
        lcs_depth = ListStrategy[lcs_strategy]([SemNode.depth(d, weights, depth_strategy) for d in self.lcs_set(other)])
        sd = SemNode.depth(self, weights, depth_strategy)
        od = SemNode.depth(other, weights, depth_strategy)
        # nlwup = self.content.wup_similarity(other.content, simulate_root=True)
        return 2*(lcs_depth)/(sd + od)
    
    def path_similarity(self, other, weights=None, depth_strategy='max', lcs_strategy='max'):
        """
        Like wupalmer, except (d(s1) + d(s2) - 2 * lcs(s1,s2))
        """
        if not issubclass(type(other), SemNode):
            other = SemNode.make(other) # this would break passing in an arbitrary maker object
        lcs_depth = ListStrategy[lcs_strategy]([SemNode.depth(d, weights, depth_strategy) for d in self.lcs_set(other)])
        sd = SemNode.depth(self, weights, depth_strategy) + self.weight(weights=weights)
        od = SemNode.depth(other, weights, depth_strategy) + other.weight(weights=weights)
        return (sd + od) - 2 * lcs_depth

In [3]:
class WordNode(SemNode):
    """
    Take a "word.pos" element as a node in the generalized hierarchy.
    """
    def resource(self):
        return "word"
    
    @property
    def name(self):
        return self.content
    
    @property
    def children(self):
        return []
    
    def word_pos(self):
        if "." in self._node:
            return self._node.split(".")
        return self._node, None
    
    @property
    def parents(self):
        """
        Lookup all TripsTypes, lookup all Wordnet Types.
        """
        w, p = self.word_pos()
        wordnet = wn.synsets(w, p)
        trips = tripsont().get_word(w, p)
        return [SemNode(c) for c in wordnet+trips]

class FakeRoot(SemNode):
    """
    FakeRoot for completeness purposes
    """
    def __init__(self):
        super(FakeRoot, self).__init__("fakeroot")
        
    def resource(self):
        return "fakeroot"
    
    @property
    def name(self):
        return "fakeroot"
    
    @property
    def parents(self):
        return []
    
    @property
    def children(self):
        return []
        

class TripsNode(SemNode):
    @property
    def name(self):
        return self.content.name
    
    @property
    def parents(self):
        return [SemNode.make(self._node.parent)]

    @property
    def children(self):
        return [SemNode.make(c) for c in self._node.children] + [SemNode.make(c) for c in self._node.wordnet]
    
    @property
    def resource(self):
        return "trips"
    
    @property
    def root(self):
        return self._node.depth == 0

    
class WordNetNode(SemNode):
    @property
    def name(self):
        return self.content.name()
    
    @property
    def parents(self):  
        # NOTE: actually this is a little bit of a problem because we're not taking
        #       WN hypernyms
        tt = tripsont()[self._node]# + self._node.hypernyms()
        if not tt:
            tt = self._node.hypernyms()
        if not tt:
            return [FakeRoot()]
        return [SemNode.make(p) for p in tt]

    @property
    def children(self):
        return [SemNode.make(c) for c in self._node.hyponyms()]
    
    @property
    def resource(self):
        return "wordnet"
    

# tests

In [4]:
# equality

cat1 = wn.synset("cat.v.1")
cat2 = wn.synset("cat.v.1")

wcat1 = WordNetNode(cat1)
wcat2 = WordNetNode(cat2)

assert issubclass(type(wcat1), SemNode)
assert wcat1 == wcat2

# hypernyms

animal = SemNode.make(tripsont()["nonhuman-animal"])
mammal = SemNode.make(tripsont()["mammal"])
assert [mammal] == animal.parents


abbess = SemNode.make(wn.synset("abbess.n.1"))
scand  = SemNode.make(wn.synset("scandinavia.n.2"))
print(abbess.paths_to_root())
print(scand.paths_to_root())

wn.synset("scandinavia.n.2").hypernyms()

2019-05-07 00:16:31,061 : pytrips : INFO : Loading ontology
2019-05-07 00:16:31,062 : root : INFO : Getting ontology.json - version 0.1.15
2019-05-07 00:16:31,356 : pytrips : INFO : Loaded ontology
2019-05-07 00:16:31,357 : pytrips : INFO : Loading lexicon
2019-05-07 00:16:31,358 : root : INFO : Getting words.json - version 0.1.15
2019-05-07 00:16:31,502 : pytrips : INFO : Loaded lexicon
2019-05-07 00:16:31,503 : pytrips : INFO : MOON18635 has 2 senses
2019-05-07 00:16:31,504 : pytrips : INFO : SHARE26081 has 2 senses
2019-05-07 00:16:31,504 : pytrips : INFO : PROD22556 has 2 senses
2019-05-07 00:16:31,505 : pytrips : INFO : CIRCUMVENT5389 has 2 senses
2019-05-07 00:16:31,506 : pytrips : INFO : FRACTURE12143 has 2 senses
2019-05-07 00:16:31,507 : pytrips : INFO : SHOW26321 has 7 senses
2019-05-07 00:16:31,508 : pytrips : INFO : PRACTICE22198 has 2 senses
2019-05-07 00:16:31,509 : pytrips : INFO : IMAGINE14629 has 2 senses
2019-05-07 00:16:31,511 : pytrips : INFO : STAIN27962 has 2 sens

2019-05-07 00:16:31,599 : pytrips : INFO : INFLAME15081 has 2 senses
2019-05-07 00:16:31,599 : pytrips : INFO : ROTATIONS24942 has 3 senses
2019-05-07 00:16:31,600 : pytrips : INFO : MEAN17908 has 2 senses
2019-05-07 00:16:31,601 : pytrips : INFO : MEAN17903 has 4 senses
2019-05-07 00:16:31,601 : pytrips : INFO : UNDO31349 has 2 senses
2019-05-07 00:16:31,602 : pytrips : INFO : LABOUR16426 has 2 senses
2019-05-07 00:16:31,603 : pytrips : INFO : CRASH7054 has 4 senses
2019-05-07 00:16:31,603 : pytrips : INFO : REGISTER23879 has 2 senses
2019-05-07 00:16:31,604 : pytrips : INFO : SKYPE26749 has 2 senses
2019-05-07 00:16:31,605 : pytrips : INFO : INCORPORATE14923 has 2 senses
2019-05-07 00:16:31,605 : pytrips : INFO : KNOW16361 has 9 senses
2019-05-07 00:16:31,607 : pytrips : INFO : DONT9233 has 2 senses
2019-05-07 00:16:31,607 : pytrips : INFO : CHITCHAT5221 has 2 senses
2019-05-07 00:16:31,608 : pytrips : INFO : TIGHTEN30112 has 2 senses
2019-05-07 00:16:31,609 : pytrips : INFO : DRIFT9

2019-05-07 00:16:31,732 : pytrips : INFO : ALTERNATIVELY1209 has 2 senses
2019-05-07 00:16:31,733 : pytrips : INFO : EXCUSE10702 has 2 senses
2019-05-07 00:16:31,734 : pytrips : INFO : INTERACT15452 has 5 senses
2019-05-07 00:16:31,734 : pytrips : INFO : DEBATE7690 has 2 senses
2019-05-07 00:16:31,735 : pytrips : INFO : GET12639 has 2 senses
2019-05-07 00:16:31,735 : pytrips : INFO : GET12633 has 13 senses
2019-05-07 00:16:31,736 : pytrips : INFO : BRUSH3896 has 2 senses
2019-05-07 00:16:31,737 : pytrips : INFO : REPAYMENTS24148 has 3 senses
2019-05-07 00:16:31,738 : pytrips : INFO : PRECIPITATE22245 has 3 senses
2019-05-07 00:16:31,738 : pytrips : INFO : FABULOUSLY11055 has 2 senses
2019-05-07 00:16:31,739 : pytrips : INFO : SHAKE26028 has 2 senses
2019-05-07 00:16:31,740 : pytrips : INFO : AREA1767 has 2 senses
2019-05-07 00:16:31,741 : pytrips : INFO : MOLT18583 has 2 senses
2019-05-07 00:16:31,741 : pytrips : INFO : UPDATE31673 has 2 senses
2019-05-07 00:16:31,742 : pytrips : INFO 

2019-05-07 00:16:31,822 : pytrips : INFO : INFORM15113 has 2 senses
2019-05-07 00:16:31,823 : pytrips : INFO : REMARK24050 has 2 senses
2019-05-07 00:16:31,824 : pytrips : INFO : REMARKS24052 has 2 senses
2019-05-07 00:16:31,824 : pytrips : INFO : ASK1945 has 6 senses
2019-05-07 00:16:31,825 : pytrips : INFO : RAP23384 has 2 senses
2019-05-07 00:16:31,825 : pytrips : INFO : AGAIN885 has 2 senses
2019-05-07 00:16:31,826 : pytrips : INFO : AFFORD854 has 3 senses
2019-05-07 00:16:31,827 : pytrips : INFO : REALISE23479 has 2 senses
2019-05-07 00:16:31,828 : pytrips : INFO : LIFT16928 has 2 senses
2019-05-07 00:16:31,829 : pytrips : INFO : LIFT16920 has 2 senses
2019-05-07 00:16:31,829 : pytrips : INFO : PREVENT22394 has 2 senses
2019-05-07 00:16:31,830 : pytrips : INFO : WORTHWHILE33230 has 4 senses
2019-05-07 00:16:31,831 : pytrips : INFO : FERMENT11359 has 2 senses
2019-05-07 00:16:31,832 : pytrips : INFO : PROMOTE22661 has 2 senses
2019-05-07 00:16:31,833 : pytrips : INFO : WHOLLY32890 

2019-05-07 00:16:31,907 : pytrips : INFO : MERGE18108 has 3 senses
2019-05-07 00:16:31,907 : pytrips : INFO : LEAD16664 has 4 senses
2019-05-07 00:16:31,908 : pytrips : INFO : LEAD16656 has 4 senses
2019-05-07 00:16:31,909 : pytrips : INFO : THEREFORE29852 has 2 senses
2019-05-07 00:16:31,909 : pytrips : INFO : BOUND3535 has 2 senses
2019-05-07 00:16:31,910 : pytrips : INFO : BIND3103 has 3 senses
2019-05-07 00:16:31,910 : pytrips : INFO : BIND3103 has 3 senses
2019-05-07 00:16:31,911 : pytrips : INFO : DIVISION9109 has 5 senses
2019-05-07 00:16:31,912 : pytrips : INFO : DRAW9372 has 2 senses
2019-05-07 00:16:31,913 : pytrips : INFO : PERFORMANCES21003 has 2 senses
2019-05-07 00:16:31,914 : pytrips : INFO : TURN-OUT31026 has 2 senses
2019-05-07 00:16:31,914 : pytrips : INFO : TURN31017 has 4 senses
2019-05-07 00:16:31,915 : pytrips : INFO : FACILITY11065 has 2 senses
2019-05-07 00:16:31,916 : pytrips : INFO : RESPOND24417 has 3 senses
2019-05-07 00:16:31,916 : pytrips : INFO : DISPERSE

...

2019-05-07 00:16:34,861 : pytrips : INFO : FORGET12044 has 7 senses
2019-05-07 00:16:34,861 : pytrips : INFO : PURIFY23023 has 3 senses
2019-05-07 00:16:34,862 : pytrips : INFO : CAUSE4666 has 2 senses
2019-05-07 00:16:34,863 : pytrips : INFO : WEAKEN32538 has 2 senses
2019-05-07 00:16:34,863 : pytrips : INFO : FREE12181 has 2 senses
2019-05-07 00:16:34,864 : pytrips : INFO : RISE24800 has 3 senses
2019-05-07 00:16:34,864 : pytrips : INFO : RISE24791 has 3 senses
2019-05-07 00:16:34,865 : pytrips : INFO : DEENERGIZE7799 has 2 senses
2019-05-07 00:16:34,866 : pytrips : INFO : THEORIZE29822 has 2 senses
2019-05-07 00:16:34,866 : pytrips : INFO : EXPLORATIONS10883 has 2 senses
2019-05-07 00:16:34,867 : pytrips : INFO : RUST25094 has 2 senses
2019-05-07 00:16:34,868 : pytrips : INFO : SOUND27380 has 3 senses
2019-05-07 00:16:34,868 : pytrips : INFO : SING26580 has 2 senses
2019-05-07 00:16:34,869 : pytrips : INFO : REPETITION24163 has 2 senses
2019-05-07 00:16:34,870 : pytrips : INFO : HUR

2019-05-07 00:16:34,940 : pytrips : INFO : ASK1945 has 6 senses
2019-05-07 00:16:34,941 : pytrips : INFO : DETERMINE8317 has 2 senses
2019-05-07 00:16:34,942 : pytrips : INFO : UNAMBITIOUS31190 has 2 senses
2019-05-07 00:16:34,942 : pytrips : INFO : ACTIVITY-VERB-DOESNOTEXIST626 has 2 senses
2019-05-07 00:16:34,943 : pytrips : INFO : NEGLECT19129 has 2 senses
2019-05-07 00:16:34,944 : pytrips : INFO : REJECT23943 has 3 senses
2019-05-07 00:16:34,944 : pytrips : INFO : BLESS3252 has 2 senses
2019-05-07 00:16:34,945 : pytrips : INFO : LOOSEN17322 has 2 senses
2019-05-07 00:16:34,945 : pytrips : INFO : KEY16181 has 2 senses
2019-05-07 00:16:34,946 : pytrips : INFO : FLY11864 has 2 senses
2019-05-07 00:16:34,947 : pytrips : INFO : EASE9672 has 3 senses
2019-05-07 00:16:34,948 : pytrips : INFO : GOSSIP12969 has 2 senses
2019-05-07 00:16:34,949 : pytrips : INFO : INDUCE15031 has 2 senses
2019-05-07 00:16:34,949 : pytrips : INFO : WANT32409 has 3 senses
2019-05-07 00:16:34,950 : pytrips : INF

2019-05-07 00:16:35,021 : pytrips : INFO : UNBIND31215 has 3 senses
2019-05-07 00:16:35,022 : pytrips : INFO : MOCK18492 has 2 senses
2019-05-07 00:16:35,023 : pytrips : INFO : PERFORMANCE21009 has 2 senses
2019-05-07 00:16:35,023 : pytrips : INFO : UNBIND31215 has 3 senses
2019-05-07 00:16:35,024 : pytrips : INFO : TAKE29266 has 10 senses
2019-05-07 00:16:35,025 : pytrips : INFO : ISOLATE15796 has 2 senses
2019-05-07 00:16:35,026 : pytrips : INFO : PROVOKE22815 has 2 senses
2019-05-07 00:16:35,027 : pytrips : INFO : CONFUSE6375 has 3 senses
2019-05-07 00:16:35,027 : pytrips : INFO : GROWTH13269 has 2 senses
2019-05-07 00:16:35,028 : pytrips : INFO : IGNITE14602 has 2 senses
2019-05-07 00:16:35,028 : pytrips : INFO : WHEN32782 has 3 senses
2019-05-07 00:16:35,029 : pytrips : INFO : HERALD13872 has 2 senses
2019-05-07 00:16:35,030 : pytrips : INFO : PERPENDICULAR21054 has 2 senses
2019-05-07 00:16:35,031 : pytrips : INFO : BOUND3535 has 2 senses
2019-05-07 00:16:35,032 : pytrips : INFO 

[[<SemNode: ont::root>, <SemNode: ont::any-sem>, <SemNode: ont::referential-sem>, <SemNode: ont::phys-object>, <SemNode: ont::natural-object>, <SemNode: ont::organism>, <SemNode: ont::animal>, <SemNode: ont::vertebrate>, <SemNode: ont::mammal>, <SemNode: ont::person>, <SemNode: Synset('abbess.n.01')>]]
[[<SemNode: 'fakeroot'>, <SemNode: Synset('scandinavia.n.02')>]]


[]

# Evaluation tools

from collections import namedtuple, Counter
from tqdm import tnrange, tqdm_notebook

Comp = namedtuple("comp", ["n1", "n2", "normal", "cross"])
def comp_string(c):
    return """
    node1: {}
    node1: {}
    wordnet only wup: {}
    cross wupalmer:   {}
    """.format(c.n1, c.n2, c.normal, c.cross)

def compare_wup(n1, n2):
    cross = SemNode.make(n1).wupalmer(SemNode.make(n2))
    normal= n1.wup_similarity(n2)
    return Comp(n1, n2, normal, cross)

res = []
for i in tqdm_notebook(range(num), desc="i"):
    for j in range(i, num):
        res.append(compare_wup(noun_sample[i], noun_sample[j]))

# count data
ranges_normal = Counter()
ranges_cross = Counter()
for i in res:
    ranges_normal[i.normal] += 1
    ranges_cross[i.cross] += 1

def binning(data, n=10):
    bin_width = 1/n
    binned = []
    for d in data:
        try:
            binned.append(int(n*d)/n)
        except:
            print(n, d, int(n*d))
    return binned

def bin_count(data, n=10):
    b_counter = Counter()
    bin_width = 1/n
    binned = []
    for d in data:
        try:
            b_counter[int(n*d)/n] += 1
        except:
            print(n, d, int(n*d))
    return b_counter

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

bins=25
data = binning([i.normal for i in res], bins)

counted = sorted(bin_count([i.normal for i in res], n=bins).items(), key=lambda x: x[0])
for c in counted:
    print(c)


plt.hist(data, density=True, bins=bins)
plt.ylabel('WordnetOnly');

data = binning([i.cross for i in res if i.cross <= 1], bins)

counted = sorted(bin_count([i.cross for i in res], n=bins).items(), key=lambda x: x[0])
for c in counted:
    print(c)

for i in res:
    if 0 > i.cross or 1 < i.cross:
        print(i)

plt.hist(data, density=True, bins=bins)
plt.ylabel('Trips-WUPalmer');

In [5]:
import pandas as pd

simlex = pd.read_csv("../SimLex-999/SimLex-999.txt", sep="\t")

In [6]:
node_weights = defaultdict(lambda: 1)

def pops(l):
    if l:
        return l[0]
    return None

def collects(l):
    return [y for y in l if y]

def n_or_v(synset):
    return synset.pos() in [wn.NOUN, wn.VERB]

def hybrid_wup(s1, s2):
    if n_or_v(s1.content) and n_or_v(s2.content):
        return s1.content.wup_similarity(s2.content)
    return s1.wupalmer(s2)

def resnik(s1, s2, ic_corpus):
    if n_or_v(s1) and n_or_v(s2):
        return s1.res_similarity(s2, ic_corpus)
    return -1 #s1.wup_similarity(s2)

def _list_fallback(func, fallback_func, args, both=False):
    result = func(*args)
    if not result or both:
        result += fallback_func(*args)
    return result

sim_strategy = {
    "mfs": lambda x, p: [SemNode.make(pops(wn.synsets(x, p)))],
    "average": lambda x, p: [SemNode.make(s) for s in wn.synsets(x, p)],
    "word": lambda x, p: [SemNode.make(v) for v in tripsont().get_word(x, p)],
    "lookup": lambda x, p: _list_fallback(
        lambda x, p: [SemNode.make(v) for v in tripsont().get_word(x, p)], 
        lambda x, p: [SemNode.make(pops(wn.synsets(x, p)))],
        [x, p]
    ),
    "lookupall": lambda x, p: _list_fallback(
        lambda x, p: [SemNode.make(v) for v in tripsont().get_word(x, p)], 
        lambda x, p: [SemNode.make(pops(wn.synsets(x, p)))],
        [x, p], both=True
    ),
    "both": lambda x, p: [SemNode.make(v) for v in tripsont().get_word(x, p)] + [SemNode.make(pops(wn.synsets(x, p)))]
}

sim_metric = {
    "cross" : lambda x, y: x.wupalmer(y, node_weights),
    "tripspath" : lambda x, y: x.path_similarity(y, weights=node_weights),
    "normal": lambda x, y: x.content.wup_similarity(y.content),
    "resnik_brown": lambda x, y: resnik(x.content, y.content, brown_ic),
    "resnik_semcor": lambda x, y: resnik(x.content, y.content, semcor_ic),
    "hybrid": hybrid_wup
}

def similarity_test(word1, word2, pos=None, metric="cross", strategy="average", select1="max", select2="max", cheats=-1):
    #return cheats
    if not pos:
        pos = "nvar"
    else:
        pos = pos.lower()[0]
    metric = sim_metric[metric]
    strategy = sim_strategy[strategy]
    results = []
    for x in pos:
        word1_node = collects(strategy(word1, x))
        word2_node = collects(strategy(word2, x))
        if word1_node and word2_node:
            scores = collects([metric(x, y) for x in word1_node for y in word2_node])
            results.append(ListStrategy[select1](scores))
    if results:
        #print("{}\t{} -> ({}) | {}".format(word1, word2, ListStrategy[select2](results), results))
        return ListStrategy[select2](results)
    return -1 # fallback of 0.5
    

In [7]:

def get_valid_scores(l1, l2):
    res1 = []
    res2 = []
    for x, y in zip(l1, l2):
        if y >= 0:
            res1.append(x)
            res2.append(y)
    print("dropping:", len(l1) - len(res1))
    return res1, res2

def confidence(rho, n):
    import math
    stderr = 1.0 / math.sqrt(n - 3)
    delta = 1.96 * stderr
    #print(rho, n, delta)
    lower = math.tanh(math.atanh(rho) - delta)
    upper = math.tanh(math.atanh(rho) + delta)
    return (lower, upper)

In [8]:
SimExperiment = namedtuple("SimExperiment", 
                           ["name", "data", "metric", "strategy", "select1", "select2", "pivot"]
                          )
SimTask = namedtuple("SimTask", ["word1", "word2", "gold", "pos"])
SimTaskResults = namedtuple("SimTaskResults", ["experiment", "instances", "spearman","pearson"])

def ws353(name, metric, strategy, select1="max", select2="max", fname="combined"):
    wordsim = pd.read_csv("../wordsim353/{}.tab".format(fname), sep="\t")
    res = []
    for i, row in wordsim.iterrows():
        res.append(SimTask(row[0], row[1], row[2], None))
    print(len(res))
    pivot=0
    if fname == "relatedness":
        pivot=0
    return SimExperiment(name, res, metric, strategy, select1, select2, pivot)

def slex(name, metric, strategy, select1="max", select2="max", fname="nvars"):
    wordsim = pd.read_csv("../SimLex-999/SimLex-999.txt".format(fname), sep="\t")
    res = []
    for i, row in wordsim.iterrows():
        if row[2].lower() in fname:
            res.append(SimTask(row[0], row[1], row[3], row[2].lower()))
    print(len(res))
    pivot=0
    if fname == "relatedness":
        pivot=0
    return SimExperiment(name, res, metric, strategy, select1, select2, pivot)

def run_experiment(exp):
    results = []
    for d in exp.data:
        r = abs(exp.pivot - similarity_test(d.word1, d.word2, 
                metric=exp.metric, 
                strategy=exp.strategy, 
                select1=exp.select1, select2=exp.select2,
                cheats=d.gold
               )) * 1/(1 - exp.pivot)
        g = d.gold
        if r >= 0:
            results.append((r,g))
    x = [r for r, g in results]
    y = [g for r, g in results]
    return SimTaskResults(exp, x, spearmanr(x, y), None)#pearsonr(x, y))

def experiment_string(exp, pandas=True, dataframe=None):
    columns = "name metric candidate choice1 choice2 instances spearmanr conf_low conf_high".split()
    if dataframe is None:
        dataframe = pd.DataFrame(columns=columns)
    e = exp.experiment
    if pandas:
        clow, chigh = confidence(exp.spearman.correlation, len(exp.instances))
        values = [e.name, e.metric, e.strategy, e.select1, e.select2, len(exp.instances),
        exp.spearman.correlation, clow, chigh] #exp.spearman.pvalue, 
        #exp.pearson[0], exp.pearson[1]]
        res = pd.DataFrame({v: [r] for r, v in zip(values, columns)})
        return dataframe.append(res)
    return """
    ---
    name:      {}
    metric:    {}
    candidate: {}
    choice:    {}/{}
    --- 
    instances:    {}
    spearman rho: {}
    p-value:      {}
    
    pearson rho:  {}
    p-value:      {}
    ================
    """.format(
        e.name, e.metric, e.strategy, e.select1, e.select2, len(exp.instances),
        exp.spearman.correlation, exp.spearman.pvalue, 
        exp.pearson[0], exp.pearson[1]
    )
    

In [9]:
# Tripswordnet
df = None
_dcache={}
fname = ["n", "v", "nv", "a"][0]

test = [ws353, slex][1]

node_weights["fakeroot"] = 1
node_weights["wordnet"] = 1
node_weights["trips"] = 1
node_weights["word"] = 1

df = experiment_string(run_experiment(test("base", "normal", strategy="average", select1="max", select2="max", fname=fname)))

df = experiment_string(run_experiment(test("Trips-Wordnet", "cross", strategy="average", select1="max", select2="max", fname=fname)), dataframe=df)

df = experiment_string(run_experiment(test("base", "normal", strategy="mfs", select1="max", select2="max", fname=fname)), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet", "cross", strategy="mfs", select1="max", select2="max", fname=fname)), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet", "cross", strategy="lookup", select1="max", select2="max", fname=fname)), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet", "cross", strategy="both", select1="max", select2="max", fname=fname)), dataframe=df)



df

666


/Users/dev/.pyenv/versions/3.5.5/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


666
666
666
666
666


,candidate,choice1,choice2,conf_high,conf_low,instances,metric,name,spearmanr
0,average,max,max,0.602150,0.496216,666,normal,base,0.551402
0,average,max,max,0.450377,0.321154,666,cross,Trips-Wordnet,0.387669
0,mfs,max,max,0.535386,0.418131,666,normal,base,0.478892
0,mfs,max,max,0.447108,0.317478,666,cross,Trips-Wordnet,0.384185
0,lookup,max,max,0.327166,0.185247,666,cross,Trips-Wordnet,0.257595
0,both,max,max,0.414114,0.280594,666,cross,Trips-Wordnet,0.349125


In [10]:
# Tripswordnet
df = None
_dcache={}
fname = ["n", "v", "nv", "a", "nvars"][1]

test = [ws353, slex][1]

node_weights["fakeroot"] = 1
node_weights["wordnet"] = 1
node_weights["trips"] = 1
node_weights["word"] = 1

df = experiment_string(run_experiment(test("base", "normal", strategy="average", select1="max", select2="max", fname=fname)))

df = experiment_string(run_experiment(test("Trips-Wordnet", "hybrid", strategy="average", select1="max", select2="max", fname=fname)), dataframe=df)

#df = experiment_string(run_experiment(test("base", "normal", strategy="mfs", select1="max", select2="max", fname=fname)), dataframe=df)

df = experiment_string(run_experiment(test("Trips-Wordnet", "cross", strategy="average", select1="max", select2="max", fname=fname)), dataframe=df)

#df = experiment_string(run_experiment(test("Trips-Wordnet", "cross", strategy="lookup", select1="max", select2="max", fname=fname)), dataframe=df)

#df = experiment_string(run_experiment(test("Trips-Wordnet", "cross", strategy="both", select1="max", select2="max", fname=fname)), dataframe=df)



df

222
222
222


,candidate,choice1,choice2,conf_high,conf_low,instances,metric,name,spearmanr
0,average,max,max,0.453098,0.220044,222,normal,base,0.341816
0,average,max,max,0.453098,0.220044,222,hybrid,Trips-Wordnet,0.341816
0,average,max,max,0.583545,0.382455,222,cross,Trips-Wordnet,0.489481
